In [1]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
raw_data = pd.read_csv("C:/Users/DIVYA/Downloads/Youtube_combined_data.csv", encoding = 'latin-1')

In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///Adaptive_playlist.db', echo=True)
sqlite_connection = engine.connect()

2020-10-15 11:18:12,120 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-10-15 11:18:12,122 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 11:18:12,125 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-10-15 11:18:12,129 INFO sqlalchemy.engine.base.Engine ()


In [4]:
sqlite_table = "youtube_scraped_data"
raw_data.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2020-10-15 11:18:12,214 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("youtube_scraped_data")
2020-10-15 11:18:12,218 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 11:18:12,223 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("youtube_scraped_data")
2020-10-15 11:18:12,225 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 11:18:12,231 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE youtube_scraped_data (
	"index" BIGINT, 
	"URL" TEXT, 
	"Title" TEXT, 
	"Description" TEXT, 
	"Tags" TEXT, 
	"Category" TEXT
)


2020-10-15 11:18:12,232 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 11:18:12,520 INFO sqlalchemy.engine.base.Engine COMMIT
2020-10-15 11:18:12,525 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_youtube_scraped_data_index ON youtube_scraped_data ("index")
2020-10-15 11:18:12,531 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 11:18:12,662 INFO sqlalchemy.engine.base.Engine COMMIT
2020-10-15 11:18:12,767 INFO sqlalchemy.engine.base.Engine BEGIN (impl

2020-10-15 11:18:15,004 INFO sqlalchemy.engine.base.Engine COMMIT


In [5]:
sqlite_connection.close()

In [ ]:
import sqlite3

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("Adaptive_playlist.db")
df = pd.read_sql_query("SELECT * from youtube_scraped_data", con)

# Verify that result of SQL query is stored in the dataframe
print(df.head())

con.close()

In [ ]:
con = sqlite3.connect("Adaptive_playlist.db")
data = pd.read_sql_query("SELECT Title,Description,Tags,Category from youtube_scraped_data", con)

In [ ]:
print(data)

In [ ]:
import spacy
import numpy as np
import pandas as pd
import re
from collections import Counter
import seaborn as sns
#from wordcloud import WordCloud

import matplotlib.pyplot as plt 
%matplotlib inline

#import mplcursors

# from os import path
# from PIL import Image


nlp = spacy.load('en_core_web_sm')

In [ ]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
#To replace nan values with dot
for i in data.columns:
    data[i] = data[i].replace(np.nan, ".")

In [ ]:
def cleaner(df1):
    "Extract relevant text from DataFrame using a regex"
    # Regex pattern for only alphanumeric, hyphenated text with 3 or more chars
    pattern = re.compile(r"[A-Za-z\-]{3,50}")
    df1['Title'] = data['Title'].str.findall(pattern).str.join(' ')
    df1['Description'] = data['Description'].str.findall(pattern).str.join(' ')
    df1['Tags'] = data['Tags'].str.findall(pattern).str.join(' ')
    #Removing numbers
#     df1['Title'] = df1['Title'].str.replace('\d+', '')
#     df1['Description'] = df1['Description'].str.replace('\d+', '')
#     df1['Tags'] = df1['Tags'].str.replace('\d+', '')
#     df1['Description'] = df1['Description'].str.replace('http\S+|www.\S+|html\S+|-\S+|com\S+|youtube\S+|https\S+','', case=False)
#     df1['Tags'] = df1['Tags'].str.replace('http\S+|www.\S+|html\S+|-\S+|com\S+|youtube\S+|https\S+','', case=False)
    return df1

In [ ]:
df_preproc = cleaner(data)

In [ ]:
df_preproc

In [ ]:
nlp_df= pd.DataFrame(columns=["Title_token", "Desc_token", "Tags_token", "Category"])
nlp_df

In [ ]:
def is_token_allowed(token):
    '''
        Only allow valid tokens which are not stop words
        and punctuation symbols.
    '''
    if (not token or not token.string.strip() or
        token.is_stop or token.is_punct):
        return False
    return True
    
def preprocess_token(token):
    # Reduce token to its lowercase lemma form
    token = token.lemma_.strip().lower()
#     token = token.replace('http\S+|www.\S+|html\S+|-\S+|com\S+|youtube\S+|https\S+','')#, case=False)
    return token

In [ ]:
def get_tokens(main_df,nlp_df):
#     nlp_df= pd.DataFrame(columns=["Title_token", "Desc_token", "Tags", "Category"])
    for i in main_df.index:
        a = {}
        row = main_df.iloc[i,].tolist()
#         print(row)
        for j in range(len(row)):
            doc_item = nlp(row[j])
            tokens = [preprocess_token(token) for token in doc_item if is_token_allowed(token)]
#             tokens_nostop = [token for token in tokens if token not in spacy_stopwords]   #No neeeddddd
            a.update({nlp_df.columns[j]:str(tokens).strip("[]")})
#             break
#             print(tokens)
#         print(a)
        nlp_df = nlp_df.append(a, ignore_index=True)
#         break
    return nlp_df

In [19]:
nlp_df =get_tokens(df_preproc,nlp_df)

In [20]:
nlp_df.head()

,Title_token,Desc_token,Tags_token,Category
0,"'husqvarna', 'svartpilen', 'vitpilen', 'review...","'husqvarna', 'svartpilen', 'husqvarna', 'vitpi...","'autocar', 'india', 'autocar', 'india', 'car',...",'automobile'
1,"'mahindra', 'thar', 'happy', 'independence', '...","'wait', 'new', 'second', 'gen', 'mahindra', 't...","'autocar', 'autocar', 'india', 'mahindra', 'th...",'automobile'
2,"'ford', 'freestyle', 'flair', 'triumph', 'stre...","'episode', 'autocar', 'india', 'quick', 'news'...","'autocar', 'india', 'autocar', 'india', 'car',...",'automobile'
3,"'track', 'attack', 'renault', 'duster', 'turbo...","'impress', 'flat', 'acceleration', 'aditya', '...","'autocar', 'india', 'autocar', 'renault', 'dus...",'automobile'
4,"'nissan', 'kicks', 'turbo', 'engine', 'deliver...","'-litre', 'turbo', 'petrol', 'engine', 'nissan...","'autocar', 'india', 'autocar', 'india', 'car',...",'automobile'


In [21]:
#Removing numbers - Added to cleaner() func
nlp_df['Title_token'] = nlp_df['Title_token'].str.replace('\d+', '')
nlp_df['Desc_token'] = nlp_df['Desc_token'].str.replace('\d+', '')
nlp_df['Tags_token'] = nlp_df['Tags_token'].str.replace('\d+', '')

In [22]:
#Removing links(http,www,com,youtube etc) - Added to cleaner() func
nlp_df['Desc_token'] = nlp_df['Desc_token'].str.replace('http\S+|www.\S+|html\S+|-\S+|com\S+|youtube\S+|https\S+','', case=False)
nlp_df['Tags_token'] = nlp_df['Tags_token'].str.replace('http\S+|www.\S+|html\S+|-\S+|com\S+|youtube\S+|https\S+','', case=False)

In [23]:
categories = nlp_df.Category.unique()
categories

array(["'automobile'", "'music'", "'entertainment'", "'education'",
       "'film_and_animation'", "'food'", "'game'", "'fashion'",
       "'news_politic'", "'pets_animal'", "'religion'",
       "'science_technology'", "'sport'", "'travel_events'",
       "'videoblogging'"], dtype=object)

In [24]:
def word_counter(text, cat, main_counts):
    freq = pd.Series(' '.join(text).split())
    freq=freq.str.cat(sep=" ")
    docs = nlp(freq)
    tokens = [preprocess_token(token) for token in docs if is_token_allowed(token)]    
    word_counts = Counter(tokens)

    counts_df = pd.DataFrame.from_dict(word_counts, orient="index").reset_index()
    counts_df = counts_df.rename(columns={"index":"Word",0:"Count"}).sort_values(by="Count", ascending=False)
    counts_df["Category"] = cat

    main_counts = main_counts.merge(counts_df, on="Word", how="outer")
    main_counts.fillna(0, inplace=True)
    main_counts["Count"] = main_counts['Count_x'] + main_counts['Count_y']
    main_counts["Category"] = cat
    main_counts.drop(["Count_x", "Count_y", "Category_x", "Category_y"], axis=1, inplace=True)
    return main_counts

#     top5 = word_counts.most_common(5)
#     return counts_df.iloc[:10,]

In [25]:
title_freq = pd.DataFrame(columns=["Category","Word", "Count"])
for category in categories:
    midman_df = pd.DataFrame(columns=["Category","Word", "Count"])
    df_cat = nlp_df[nlp_df.Category==category] 
#     freq = pd.Series(' '.join(df_cat['Title_token']).split()).value_counts()[:5]
#     print(freq)
    title_freq = title_freq.append(word_counter(df_cat.Title_token, category,midman_df), ignore_index=True)
#     input("Enter any key to continue")

In [26]:
len(title_freq.Category.unique())

15

In [27]:
title_freq.to_csv("C:\\Users\\Divya\\Documents\\Title_counts.csv", index=False)

In [28]:
engine = create_engine('sqlite:///Adaptive_playlist.db', echo=True)
sqlite_connection = engine.connect()
sqlite_table = "Title_counts"
title_freq.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2020-10-06 15:30:12,734 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-10-06 15:30:12,985 INFO sqlalchemy.engine.base.Engine ()
2020-10-06 15:30:13,022 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-10-06 15:30:13,025 INFO sqlalchemy.engine.base.Engine ()
2020-10-06 15:30:13,337 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Title_counts")
2020-10-06 15:30:13,341 INFO sqlalchemy.engine.base.Engine ()
2020-10-06 15:30:13,434 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Title_counts")
2020-10-06 15:30:13,439 INFO sqlalchemy.engine.base.Engine ()
2020-10-06 15:30:13,568 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Title_counts" (
	"index" BIGINT, 
	"Category" TEXT, 
	"Word" TEXT, 
	"Count" BIGINT
)


2020-10-06 15:30:13,569 INFO sqlalchemy.engine.base.Engine ()
2020-10-06 15:30:13,802 INFO sqlalchemy.engine.base.Engine COMMIT
2020-10-06 15:30:13,805 INFO

In [29]:
sqlite_connection.close()

In [30]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("Adaptive_playlist.db")
title_data = pd.read_sql_query("SELECT * from Title_counts", con)

# Verify that result of SQL query is stored in the dataframe
print(title_data.head())

con.close()

   index      Category        Word  Count
0      0  'automobile'  powerdrift   1610
1      1  'automobile'       india    809
2      2  'automobile'     autocar    650
3      3  'automobile'      review    300
4      4  'automobile'        auto    236


In [ ]:
desc_freq= pd.DataFrame(columns=["Category","Word", "Count"])
for category in categories:
    midman_df = pd.DataFrame(columns=["Category","Word", "Count"])
    df_cat = nlp_df[nlp_df.Category==category]
    for i in range(0,df_cat.shape[0],200):
        if (i+200)<df_cat.shape[0]:l=i+200
        else: l = df_cat.shape[0]+1            
        midman_df = word_counter(df_cat.Desc_token[i:l], category, midman_df)
#         break
    midman_df.sort_values(by='Count', ascending=False, inplace=True)
#     plot_unigram(midman_df[:20], category)
    desc_freq = desc_freq.append(midman_df, ignore_index=True)
#     desc_freq.update({category: word_counter(df_cat.Desc_token)})
#     input("Enter any key to continue")

In [ ]:
len(desc_freq.Category.unique())

In [2]:
desc = pd.read_csv("C:\\Users\\DIVYA\\Documents\\Desc_counts.csv", index=False)

NameError: name 'pd' is not defined

In [ ]:
engine = create_engine('sqlite:///Adaptive_playlist.db', echo=True)
sqlite_connection = engine.connect()
sqlite_table = "Desc_counts"
desc_freq.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

In [ ]:
sqlite_connection.close()

In [ ]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("Adaptive_playlist.db")
desc_data = pd.read_sql_query("SELECT * from Desc_counts", con)

# Verify that result of SQL query is stored in the dataframe
print(desc_data.head())

con.close()

In [ ]:
tags_freq=pd.DataFrame(columns=["Category","Word", "Count"])

for category in categories:
    midman_df = pd.DataFrame(columns=["Category","Word", "Count"])
    df_cat = nlp_df[nlp_df.Category==category] 
    
    for i in range(0,df_cat.shape[0],200):
        if (i+200)<df_cat.shape[0]:l=i+200
        else: l = df_cat.shape[0]+1            
        midman_df = word_counter(df_cat.Tags_token[i:l], category, midman_df)
#         break
    midman_df.sort_values(by='Count', ascending=False, inplace=True)
#     plot_unigram(midman_df[:20], category)
    tags_freq = tags_freq.append(midman_df, ignore_index=True)

#     tags_freq = tags_freq.append(word_counter(df_cat.Tags_token, category), ignore_index=True)
#     tags_freq.update({category: word_counter(df_cat.Tags_token)})
#     input("Enter any key to continue")

In [ ]:
len(tags_freq.Category.unique())

In [ ]:
tags_freq.to_csv("C:\\Users\\DIVYA\\Documents\\Tags_counts.csv", index=False)

In [ ]:
engine = create_engine('sqlite:///Adaptive_playlist.db', echo=True)
sqlite_connection = engine.connect()
sqlite_table = "Tags_counts"
tags_freq.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

In [ ]:
sqlite_connection.close()

In [ ]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("Adaptive_playlist.db")
tags_data = pd.read_sql_query("SELECT * from Tags_counts", con)

# Verify that result of SQL query is stored in the dataframe
print(tags_data.head())

con.close()

In [ ]:
nlp_df.to_csv("C:\\Users\\DIVYA\\Documents\\Tokenised_data.csv")